# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import keras

In [2]:
from keras.layers import Convolution2D, Conv2D
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MultiLabelBinarizer
from keras.layers.normalization import BatchNormalization
from keras.layers import MaxPooling2D, AveragePooling2D
from keras.layers import Flatten
from keras.layers import Dense, Activation
from keras import regularizers
from keras.optimizers import Adam
from keras.layers import Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
import pandas_profiling

***GPU***

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7676065243757619997
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3181065012
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11585613860143807681
physical_device_desc: "device: 0, name: GeForce 940MX, pci bus id: 0000:01:00.0, compute capability: 5.0"
]


In [4]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [5]:
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image

# Custom Model

In [6]:
from keras.models import Sequential
from keras.callbacks import CSVLogger

model = Sequential()
model.add(Conv2D(32, (5, 5), strides=(2,2), use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', input_shape = (324, 324, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))

#layer 2
model.add(Conv2D(64, (5, 5), strides=(2,2), use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))

#layer 3
model.add(Conv2D(128, (5, 5), strides=(1,1), use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros'))
model.add(Activation('relu'))

#layer 4
model.add(Conv2D(256, (5, 5), strides=(1,1), use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros'))
model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size=(2, 2),strides=(1, 1)))

#flatten
model.add(Flatten(input_shape=(11, 11)))

#dense layer Classifier
model.add(Dense(512))
model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones'))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(126))
model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones'))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(30))
model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones'))
model.add(Activation('softmax'))

In [7]:
opt = Adam(lr=3e-4, decay=(3e-4)/ 25)
# distribution
model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])

In [8]:
#data augmentation for training
train_datagen = ImageDataGenerator(rescale=1. / 255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

#data augmentation for testing:
valid_datagen = ImageDataGenerator(rescale=1. / 255)

#take path to dir and generate batches of augmented data
train_generator = train_datagen.flow_from_directory("C:/Users/nandish/Desktop/Segmented Medicinal Leaf Images/Train", target_size=(324, 324), batch_size=64, class_mode='categorical')

validation_generator = valid_datagen.flow_from_directory("C:/Users/nandish/Desktop/Segmented Medicinal Leaf Images/Test", target_size=(324, 324), batch_size=64, class_mode='categorical')

csv_logger = CSVLogger('leaf_v1-1.csv');

#model training
model.fit(
    train_generator,
    steps_per_epoch=1422 // 64,
    epochs=100,
    validation_data=validation_generator,
    validation_steps=413 // 64,
    callbacks=[csv_logger])

model.save("C:/Users/nandish/Desktop/Segmented Medicinal Leaf Images/savedModel.h5")

Found 1422 images belonging to 30 classes.
Found 413 images belonging to 30 classes.
Epoch 1/100
22/22 [==============================] - 92s 4s/step - loss: 3.4217 - accuracy: 0.0755 - val_loss: 3.7951 - val_accuracy: 0.0260
Epoch 2/100
22/22 [==============================] - 79s 4s/step - loss: 2.8033 - accuracy: 0.2014 - val_loss: 3.5598 - val_accuracy: 0.0729
Epoch 3/100
22/22 [==============================] - 79s 4s/step - loss: 2.6564 - accuracy: 0.2524 - val_loss: 3.1490 - val_accuracy: 0.1198
Epoch 4/100
22/22 [==============================] - 84s 4s/step - loss: 2.5084 - accuracy: 0.3034 - val_loss: 2.9406 - val_accuracy: 0.1745
Epoch 5/100
22/22 [==============================] - 80s 4s/step - loss: 2.3364 - accuracy: 0.3883 - val_loss: 3.0456 - val_accuracy: 0.1589
Epoch 6/100
22/22 [==============================] - 79s 4s/step - loss: 2.2513 - accuracy: 0.4170 - val_loss: 2.9826 - val_accuracy: 0.1693
Epoch 7/100
22/22 [==============================] - 80s 4s/step - lo

22/22 [==============================] - 80s 4s/step - loss: 0.5256 - accuracy: 0.9861 - val_loss: 0.4407 - val_accuracy: 0.9297
Epoch 59/100
22/22 [==============================] - 79s 4s/step - loss: 0.5237 - accuracy: 0.9737 - val_loss: 0.5034 - val_accuracy: 0.9323
Epoch 60/100
22/22 [==============================] - 80s 4s/step - loss: 0.5092 - accuracy: 0.9781 - val_loss: 0.4697 - val_accuracy: 0.9427
Epoch 61/100
22/22 [==============================] - 84s 4s/step - loss: 0.5412 - accuracy: 0.9705 - val_loss: 0.5564 - val_accuracy: 0.9141
Epoch 62/100
22/22 [==============================] - 81s 4s/step - loss: 0.4906 - accuracy: 0.9713 - val_loss: 0.4174 - val_accuracy: 0.9531
Epoch 63/100
22/22 [==============================] - 80s 4s/step - loss: 0.5038 - accuracy: 0.9743 - val_loss: 0.4542 - val_accuracy: 0.9297
Epoch 64/100
22/22 [==============================] - 81s 4s/step - loss: 0.4842 - accuracy: 0.9766 - val_loss: 0.4280 - val_accuracy: 0.9505
Epoch 65/100
22/22 